# Population Dataset Ingest

In [17]:
import numpy as np
import pandas as pd
import re
import sys
import locale

locale_ingest_str = 'es_ES.UTF-8'
locale.setlocale(locale.LC_ALL, locale_ingest_str)

date_format_ingest_raw = '%d de %B de %Y'
date_format_ingest_std = '%Y-%m-%d'

## Dataset Description

Número de habitantes en España para cada mes desde el inicio de la serie histórica, desagregados por sexo, provincia y edad.

## Raw Dataset Generation

There's no API to retreive the data from INE, so this process must be followed manually:

1. Acceder a página del INE sobre ["Población residente por fecha, sexo y edad"](https://www.ine.es/jaxiT3/Tabla.htm?t=31304).
    
2. En la sección de **Seleccione valores a consulta**:
- Para la variable *Sexo*, seleccionar todos los valores pulsando sobre el icono ![Seleccionar todos](../assets/images/iconoselecciontodos.gif).
- Para la variable *Edad*, seleccionar todos los valores pulsando sobre el icono ![Seleccionar todos](../assets/images/iconoselecciontodos.gif).
- Para la variable *Provincias*, seleccionar todos los valores pulsando sobre el icono ![Seleccionar todos](../assets/images/iconoselecciontodos.gif).
- Para la variable *Periodo*, seleccionar todos los valores pulsando sobre el iconode ![Seleccionar todos](../assets/images/iconoselecciontodos.gif).

3. Pulsar sobre el icono ![Descargar](../assets/images/descargar_borde_24x24.gif) y seleccionar la opción *CSV: separado por ;*.

4. Una vez descargado el fichero con el dataset Raw, renombrar el fichero como `Population.csv` y situarlo en el subdirectorio `data/raw`.

## Standard Dataset Generation

Leemos el dataset RAW.

In [2]:
# Columns names for renaming.
column_names = ['sex_str', 'age_str', 'province_str', 'date_str', 'total_str']

# Raw Datased load.
df_population = pd.read_csv('....//data/raw/Population.csv', encoding = 'utf-8', sep = ';', header = 0, names = column_names, quotechar = "\"", na_filter = False, low_memory = False)

In [3]:
df_population.head()

,sex_str,age_str,province_str,date_str,total_str
0,Ambos sexos,Total,Total Nacional,1 de julio de 2019,47.100.396
1,Ambos sexos,Total,Total Nacional,1 de enero de 2019,46.937.060
2,Ambos sexos,Total,Total Nacional,1 de julio de 2018,46.728.814
3,Ambos sexos,Total,Total Nacional,1 de enero de 2018,46.658.447
4,Ambos sexos,Total,Total Nacional,1 de julio de 2017,46.532.869


In [4]:
# Detete from dataset rows with total values for every variable column.

df_population.drop(df_population[df_population['sex_str'] == 'Ambos sexos'].index , inplace = True)
df_population.drop(df_population[df_population['age_str'] == 'Total'].index , inplace = True)
df_population.drop(df_population[df_population['province_str'] == 'Total Nacional'].index , inplace = True)

In [5]:
df_population.head()

,sex_str,age_str,province_str,date_str,total_str
540274,Hombres,0 años,02 Albacete,1 de julio de 2019,1.494
540275,Hombres,0 años,02 Albacete,1 de enero de 2019,1.584
540276,Hombres,0 años,02 Albacete,1 de julio de 2018,1.641
540277,Hombres,0 años,02 Albacete,1 de enero de 2018,1.694
540278,Hombres,0 años,02 Albacete,1 de julio de 2017,1.689


In [6]:
# Delete agregated rows when 'age_str' column contains a super-agregation value
df_population.drop(df_population[df_population['age_str'] == '85 y más años'].index , inplace = True)

In [7]:
# Defining data conversion and extraction functions

# Sex conversion function
def data_convert_sex(value_str):
    if value_str == 'Hombres':
        return 'M'
    else:
        if value_str == 'Mujeres':
            return 'F'
    return None

# Age extraction function 
def data_extract_age(value_str):
    m = re.match("(\d+)([^\d]+)?$", value_str.strip())
    if m and len(m.groups()) == 2:
        return m.groups()[0]
    return None

# Province extraction function
def data_extract_province(value_str):
    m = re.match("(\d+)([^\d]+)?$", value_str.strip())
    if m and len(m.groups()) == 2:
        return m.groups()[0]
    return None
    
# Data conversion function
def data_convert_date(value_str, date_format_origin, date_format_target):
    try:
        return pd.to_datetime(str(value_str), format = date_format_origin).strftime(date_format_target)
    except:
        return None

# Total conversion function
def data_convert_total(value_str):
    if value_str == '':
        return "0"
    try:
        return str(locale.atoi(str(value_str)))
    except:
        return None

In [8]:
# Applying conversion and extraction functions to dataset.

df_population['sex'] = df_population['sex_str'].apply(lambda x: data_convert_sex(x))
df_population['age'] = df_population['age_str'].apply(lambda x: data_extract_age(x))
df_population['province'] = df_population['province_str'].apply(lambda x: data_extract_province(x))
df_population['date'] = df_population['date_str'].apply(lambda x: data_convert_date(x, date_format_ingest_raw, date_format_ingest_std))
df_population['total'] = df_population['total_str'].apply(lambda x: data_convert_total(x))

In [9]:
# None values mark rows with standarization problems.
# Check for None values in all columns.

# Erros on sex column.
sex_ko_count = len(df_population[df_population['sex'] == None])
if sex_ko_count != 0: 
    sys.exit('Found {0} rows with incorrect values on \'sex_str\' column.'.format(sex_ko_count))

# Erros on age column.
age_ko_count = len(df_population[df_population['age'] == None])
if age_ko_count != 0: 
    sys.exit('Found {0} rows with incorrect values on \'age_str\' column.'.format(age_ko_count))

# Erros on province column.
province_ko_count = len(df_population[df_population['province'] == None])
if province_ko_count != 0: 
    sys.exit('Found {0} rows with incorrect values on \'province_str\' column.'.format(province_ko_count))
    
# Erros on date column.
date_ko_count = len(df_population[df_population['date'] == None])
if date_ko_count != 0: 
    sys.exit('Found {0} rows with incorrect values on \'date_str\' column.'.format(date_ko_count))
    
# Erros on total column.
total_ko_count = len(df_population[df_population['total'] == None])
if total_ko_count != 0: 
    sys.exit('Found {0} rows with incorrect values on \'total_str\' column.'.format(total_ko_count))

In [10]:
df_population.head()

,sex_str,age_str,province_str,date_str,total_str,sex,age,province,date,total
540274,Hombres,0 años,02 Albacete,1 de julio de 2019,1.494,M,0,02,2019-07-01,1494
540275,Hombres,0 años,02 Albacete,1 de enero de 2019,1.584,M,0,02,2019-01-01,1584
540276,Hombres,0 años,02 Albacete,1 de julio de 2018,1.641,M,0,02,2018-07-01,1641
540277,Hombres,0 años,02 Albacete,1 de enero de 2018,1.694,M,0,02,2018-01-01,1694
540278,Hombres,0 años,02 Albacete,1 de julio de 2017,1.689,M,0,02,2017-07-01,1689


In [11]:
# Applying type conversion functions to dataset.

df_population['sex'] = df_population['sex'].astype(str)
df_population['age'] = df_population['age'].apply(locale.atoi)
df_population['province'] = df_population['province'].apply(locale.atoi)
df_population['date'] = df_population['date'].apply(lambda x: pd.to_datetime(x, format = date_format_ingest_std))
df_population['total'] = df_population['total'].apply(locale.atoi)

In [12]:
df_population.head()

,sex_str,age_str,province_str,date_str,total_str,sex,age,province,date,total
540274,Hombres,0 años,02 Albacete,1 de julio de 2019,1.494,M,0,2,2019-07-01,1494
540275,Hombres,0 años,02 Albacete,1 de enero de 2019,1.584,M,0,2,2019-01-01,1584
540276,Hombres,0 años,02 Albacete,1 de julio de 2018,1.641,M,0,2,2018-07-01,1641
540277,Hombres,0 años,02 Albacete,1 de enero de 2018,1.694,M,0,2,2018-01-01,1694
540278,Hombres,0 años,02 Albacete,1 de julio de 2017,1.689,M,0,2,2017-07-01,1689


In [13]:
# Remove Raw Columns

df_population.drop(column_names, axis = 1, inplace = True)

In [14]:
df_population.head()

,sex,age,province,date,total
540274,M,0,2,2019-07-01,1494
540275,M,0,2,2019-01-01,1584
540276,M,0,2,2018-07-01,1641
540277,M,0,2,2018-01-01,1694
540278,M,0,2,2017-07-01,1689


In [16]:
# Standard Dataset saving

df_population.to_csv('../../data/standard/Population.csv', index = False)